# 持久化 RAG 系統的完整實戰教學

本文是一份工程導向的完整教學稿，目標不是只「跑得動」，而是讓你真正理解：

為什麼這樣設計索引

>- 為什麼 persistence 會出錯
>- 為什麼 Retrieval 策略需要隨資料規模演進
>- 本文將逐步建構一個 可持久化（Persistent）、可擴展（Scalable）、避免常見陷阱 的 RAG 系統。

## 0️⃣ 前言：我們要解決什麼問題？

在實務上，RAG 系統常見三個痛點：

>- 索引與 Embedding 維度不一致，導致查詢時崩潰
>- 向量資料明明存在磁碟，卻怎麼查都查不到
>- 文件一多，單一相似度搜尋開始失準

![caption](Gemini_Generated_Image_pzkcrgpzkcrgpzkc.png)

這三個問題，其實都來自同一件事： RAG 系統是一個「狀態型系統（stateful system）」，而不是一次性腳本。

本文將用 LlamaIndex + FAISS/Qdrant，從資料蒐集一路走到 Advanced Retrieval，並在每一個關鍵點說清楚「為什麼」。

## 1️⃣ 資料準備與文件建模（Document Layer）
為什麼這一步重要？

RAG 系統的品質，80% 來自於 資料建模是否合理。如果一開始文件與 metadata 設計錯誤，後面的 embedding、retrieval 都只是在放大錯誤。

### 資料來源選擇

我們使用 Wikipedia（中文）作為資料來源，原因是：

>- 內容長度中等（適合 chunking 示範）
>- 結構鬆散（接近真實世界資料）
>- 主題明確（方便驗證 retrieval 正確性）

### Metadata 設計

每份文件都會附加 metadata，例如：

- author：漫畫作者

為什麼要這樣做？

Metadata 並不是裝飾品，而是：

>- Retrieval 階段的「搜尋空間約束條件」
>- 未來進行 metadata filtering / auto-retrieval 的基礎

In [ ]:
from llama_index.readers.file import FlatReader
from llama_index.core import SimpleDirectoryReader#, SummaryIndex

In [ ]:
wiki_titles = ["鋼之鍊金術師", "一拳超人", "ONE_PIECE", "東京喰種"]
wiki_metadatas = {
    "鋼之鍊金術師": {
        "author": "荒川弘",
    },
    "一拳超人": {
        "author": "ONE",
    },
    "ONE_PIECE": {
        "author": "尾田榮一郎",
    },
    "東京喰種": {
        "author": "石田翠",
    },
}

In [ ]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia(user_agent='AI Tutorial(mengchiehling@gmail.com)', language='zh-tw')

In [ ]:
from pathlib import Path

for title in wiki_titles:
    page = wiki_wiki.page(title)
    wiki_text = page.text

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(wiki_text)

In [ ]:
from glob import glob

docs = SimpleDirectoryReader(input_files=[f for f in glob("data/*.txt")]).load_data()

for doc in docs:
    comic_name = Path(doc.metadata['file_name']).stem
    doc.metadata.update(wiki_metadatas[comic_name])

In [ ]:
docs[0].metadata

In [ ]:
import os

os.chdir("../")

from initialization import credential_init

credential_init()

建立nodes

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=128,
    chunk_overlap=0,
    separators=["\n\n", "\n", "。", "，", " "]
).split_text

semantic_splitter_node_parser = SemanticSplitterNodeParser.from_defaults(
    embed_model = embed_model,
    sentence_splitter=text_splitter,
    include_metadata=True,
    include_prev_next_rel=True,
    breakpoint_percentile_threshold=60,
)

raw_nodes = semantic_splitter_node_parser.get_nodes_from_documents(docs)

## FAISS + VectorStoreIndex Persist

- 建立 embedding
- 建立 vectorstore
- 建立 storage context
- 建立 vectorstoreindex

In [ ]:
import faiss
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.faiss import FaissVectorStore

persist_dir = "./week_2/storage_faiss"

# Overwrite logic: remove the folder if it exists
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
)

In [ ]:
from src.ollama_connection import llama_index_ollama

ollama_llm = llama_index_ollama(model="gpt-oss:120b-cloud", temperature=0)

In [ ]:
faiss_index = VectorStoreIndex(
    raw_nodes,
    storage_context=storage_context,
    embed_model=embed_model,
)

In [ ]:
faiss_query_engine = faiss_index.as_query_engine(similarity_top_k=5,
                                                 response_mode="compact",
                                                 llm=ollama_llm)

In [ ]:
output = faiss_query_engine.query("誰是最強的光頭?")

In [ ]:
output.response

儲存 Storage

In [ ]:
storage_context.persist(persist_dir="./week_2/storage_faiss")

讀取 Storage 重建 Index

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

# 【步驟 A】從本地路徑還原向量存儲物件
# 注意：不要手動建立空的 faiss.IndexFlatL2，直接從 persist_dir 讀取
vector_store = FaissVectorStore.from_persist_dir("./week_2/storage_faiss")

# 【步驟 B】構建存儲上下文
storage_context_loaded = StorageContext.from_defaults(
    vector_store=vector_store,
    persist_dir="./week_2/storage_faiss"
)

# 【步驟 C】載入索引並建立查詢引擎
faiss_vector_index_loaded = load_index_from_storage(
    storage_context_loaded,
    embed_model=embed_model
)

faiss_query_engine_loaded = faiss_vector_index_loaded.as_query_engine(
    similarity_top_k=2,
    embed_model=embed_model,
    llm=ollama_llm
)

In [ ]:
output = faiss_query_engine.query("誰是最強的光頭?")

In [ ]:
output.response

### 更新 Storage Context

In [ ]:
title = "银之匙_Silver_Spoon"

page = wiki_wiki.page(title)
wiki_text = page.text

In [ ]:
from llama_index.core import Document

new_doc = Document(text=wiki_text, metadata={"author": "荒川弘"})

new_nodes = semantic_splitter_node_parser.get_nodes_from_documents([new_doc])

In [ ]:
# 確保 nodes 已經被向量化
for node in new_nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [ ]:
# new_nodes[0]

更新 vectorstore index

In [ ]:
faiss_vector_index_loaded.insert_nodes(new_nodes)

In [ ]:
faiss_query_engine_updated = faiss_vector_index_loaded.as_query_engine(
    similarity_top_k=2,
    embed_model=embed_model,
    llm=ollama_llm
)

In [ ]:
output = faiss_query_engine_updated.query("銀之匙的創作背景")

In [ ]:
output.response

In [ ]:
faiss_vector_index_loaded.storage_context.persist(persist_dir="./week_2/storage_faiss_new")

實驗是否可以正常執行

In [ ]:
vector_store = FaissVectorStore.from_persist_dir("./week_2/storage_faiss_new")

storage_context_loaded = StorageContext.from_defaults(
    vector_store=vector_store,
    persist_dir="./week_2/storage_faiss_new"
)

faiss_vector_index_loaded = load_index_from_storage(
    storage_context_loaded,
    embed_model=embed_model
)

faiss_query_engine_loaded = faiss_vector_index_loaded.as_query_engine(
    similarity_top_k=2,
    embed_model=embed_model,
    llm=ollama_llm
)

In [ ]:
output = faiss_query_engine_loaded.query("銀之匙的創作背景")

In [ ]:
output.response

### Shared Storage Context 與 Multi-Index 管理（進階）

每次呼叫 VectorStoreIndex.from_documents，LlamaIndex 都會：

- 先建立一個臨時 UUID index
- 再讓你手動指定 index_id

如果沒有清理，就會：

- 多存一個「沒人用的 index」

解法核心觀念

- StorageContext 是 索引一致性的邊界
- index_id 必須同步更新到 index_store

In [ ]:
import shutil

persist_dir = "./week_2/storage_shared"

# Overwrite logic: remove the folder if it exists
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
)

# add documents to index
docs = SimpleDirectoryReader(input_files=[f for f in glob("week_2/data/*.txt")]).load_data()

for doc in docs:
    comic_name = Path(doc.metadata['file_name']).stem
    doc.metadata.update(wiki_metadatas[comic_name])

    nodes = semantic_splitter_node_parser.get_nodes_from_documents([doc])
    
    index = VectorStoreIndex(
        nodes,
        storage_context=storage_context,
        embed_model=embed_model,
    )
    # 2. To strictly clean up the auto-generated one:
    old_id = index.index_id
    print(old_id)
    # Manually set the index_id on the index_struct
    index.set_index_id(comic_name)
    # index.index_struct.index_id = wiki_title
    
    # Sync the index_id back to the storage context's index store
    storage_context.index_store.add_index_struct(index.index_struct)

    # FIX: Use a try-except or check against the dictionary keys correctly
    # Usually, storage_context.index_store.index_structs is the internal dict
    try:
        storage_context.index_store.delete_index_struct(old_id)
    except Exception:
        # If it's already gone or doesn't exist, just move on
        pass

# Persist ONCE after the loop (more efficient than inside the loop)
storage_context.persist(persist_dir=persist_dir)

讀取指定index_id

In [ ]:
vector_store_loaded = FaissVectorStore.from_persist_dir("./week_2/storage_shared")

storage_context = StorageContext.from_defaults(
    vector_store=vector_store_loaded,
    persist_dir="./week_2/storage_shared",
)

In [ ]:
vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "東京喰種",
    embed_model=embed_model
)

query_engine_tokyo_ghoul = vector_index_tokyo_ghoul.as_query_engine(
    similarity_top_k=2,
    llm=ollama_llm,
)

In [ ]:
# query_engine_tokyo_ghoul.query("解釋赫子")

### 加入新的index

In [ ]:
# storage_context.index_store.index_structs()

In [ ]:
from llama_index.core import VectorStoreIndex

# 建立新的index
new_index = VectorStoreIndex(
    nodes=new_nodes,               # 你要加入的新資料
    storage_context=storage_context,
    embed_model=embed_model,
)

old_id = new_index.index_id
new_index.set_index_id("銀之匙")

# Sync the index_id back to the storage context's index store
storage_context.index_store.add_index_struct(index.index_struct)

storage_context.index_store.delete_index_struct(old_id)

In [ ]:
storage_context.persist(persist_dir="./week_2/storage_shared_new")

### 更新舊的index

In [ ]:
title = "石田翠"

page = wiki_wiki.page(title)
wiki_text = page.text

new_doc = Document(text=wiki_text)

tokyo_ghoul_new_nodes = semantic_splitter_node_parser.get_nodes_from_documents([new_doc])

In [ ]:
# 確保 nodes 已經被向量化
for node in tokyo_ghoul_new_nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [ ]:
vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "東京喰種",
    embed_model=embed_model
)

In [ ]:
vector_index_tokyo_ghoul.insert_nodes(tokyo_ghoul_new_nodes)
storage_context.persist(persist_dir="./week_2/storage_shared_tokyo_ghoul_new")

檢查

In [ ]:
vector_store_loaded = FaissVectorStore.from_persist_dir("./week_2/storage_shared_tokyo_ghoul_new")

storage_context = StorageContext.from_defaults(
    vector_store=vector_store_loaded,
    persist_dir="./week_2/storage_shared_tokyo_ghoul_new",
)

vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "東京喰種",
    embed_model=embed_model
)

query_engine_tokyo_ghoul = vector_index_tokyo_ghoul.as_query_engine(
    similarity_top_k=5,
    llm=ollama_llm,
)

In [ ]:
# query_engine_tokyo_ghoul.query("東京喰種作者的人生經歷")

## Qdrant + VectorStoreIndex Persist

In [ ]:
await client.close()

In [ ]:
from qdrant_client import AsyncQdrantClient
from llama_index.vector_stores.qdrant import QdrantVectorStore

"""
await client.close()
"""

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

persist_dir = "./week_2/storage_qdrant"

# Overwrite logic: remove the folder if it exists
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

storage_context = StorageContext.from_defaults(
    vector_store=qdrant_vector_store,
)

In [ ]:
qdrant_index = VectorStoreIndex(
    raw_nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    llm=ollama_llm,
    use_async=True
)

In [ ]:
storage_context.persist(persist_dir="./week_2/storage_qdrant")

測試還原 qdrant_index

In [ ]:
await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    persist_dir="./week_2/storage_shared_qdrant",
    vector_store=qdrant_vector_store,
)

In [ ]:
# 4. 重建 index
qdrant_vector_index = load_index_from_storage(
    storage_context=storage_context,
    embed_model=embed_model,
    llm=ollama_llm,
)

In [ ]:
qdrant_query_engine = qdrant_index.as_query_engine(similarity_top_k=5,
                                                   llm=ollama_llm)

In [ ]:
# await qdrant_query_engine.aquery("海賊王最新劇情")

### 更新Storage Context

In [ ]:
title = "银之匙_Silver_Spoon"

page = wiki_wiki.page(title)
wiki_text = page.text

In [ ]:
new_doc = Document(text=wiki_text, metadata={"author": "荒川弘"})

new_nodes = semantic_splitter_node_parser.get_nodes_from_documents([new_doc])

In [ ]:
# 確保 nodes 已經被向量化
for node in new_nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [ ]:
# async version
await qdrant_index.ainsert_nodes(new_nodes)

In [ ]:
qdrant_query_engine = qdrant_index.as_query_engine(similarity_top_k=10,
                                                   llm=ollama_llm)

output = await qdrant_query_engine.aquery("銀之匙的創作背景")

In [ ]:
output.response

In [ ]:
storage_context.persist()

測試Storage Context是否被成功persist

In [ ]:
await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    persist_dir="./week_2/storage_qdrant",
    vector_store=qdrant_vector_store,
)

qdrant_vector_index = load_index_from_storage(
    storage_context=storage_context,
    embed_model=embed_model,
    llm=ollama_llm,
)

qdrant_query_engine = qdrant_vector_index.as_query_engine(similarity_top_k=10,
                                                   llm=ollama_llm)

output = await qdrant_query_engine.aquery("銀之匙的創作背景")

In [ ]:
output.response

### Shared Storage Context 與 Multi-Index 管理（進階）

In [ ]:
persist_dir = "./week_2/storage_qdrant_shared"

# Overwrite logic: remove the folder if it exists
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant_shared")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    vector_store=qdrant_vector_store,
)

# add documents to index
docs = SimpleDirectoryReader(input_files=[f for f in glob("week_2/data/*.txt")]).load_data()

for doc in docs:
    comic_name = Path(doc.metadata['file_name']).stem
    doc.metadata.update(wiki_metadatas[comic_name])

    nodes = semantic_splitter_node_parser.get_nodes_from_documents([doc])
    
    index = VectorStoreIndex(
        nodes,
        storage_context=storage_context,
        embed_model=embed_model,
        use_async=True
    )

    old_id = index.index_id
    print(old_id)
    index.set_index_id(comic_name)

    storage_context.index_store.add_index_struct(index.index_struct)

    try:
        storage_context.index_store.delete_index_struct(old_id)
    except Exception:
        pass

# Persist ONCE after the loop (more efficient than inside the loop)
storage_context.persist(persist_dir=persist_dir)

In [ ]:
await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant_shared")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    persist_dir="./week_2/storage_qdrant_shared",
    vector_store=qdrant_vector_store,
)

vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "東京喰種",
    embed_model=embed_model
)

query_engine_tokyo_ghoul = vector_index_tokyo_ghoul.as_query_engine(
    similarity_top_k=10,
    llm=ollama_llm,
)

output = await query_engine_tokyo_ghoul.aquery("東京喰種作者的人生經歷")

In [ ]:
output.response

#### 加入新的index

In [ ]:
# 建立新的index
new_index = VectorStoreIndex(
    nodes=new_nodes,               # 你要加入的新資料
    storage_context=storage_context,
    embed_model=embed_model,
    use_async=True
)

old_id = new_index.index_id
new_index.set_index_id("銀之匙")

# Sync the index_id back to the storage context's index store
storage_context.index_store.add_index_struct(new_index.index_struct)

storage_context.index_store.delete_index_struct(old_id)

storage_context.persist(persist_dir="./week_2/storage_qdrant_shared_new")

In [ ]:
await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant_shared")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    persist_dir="./week_2/storage_qdrant_shared_new",
    vector_store=qdrant_vector_store,
)

vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "銀之匙",
    embed_model=embed_model
)

query_engine_tokyo_ghoul = vector_index_tokyo_ghoul.as_query_engine(
    similarity_top_k=10,
    llm=ollama_llm,
)

output = await query_engine_tokyo_ghoul.aquery("銀之匙的創作背景")

In [ ]:
output.response

#### 更新Index

就留作練習吧~~

# 進階檢索策略 - Part 1

## Small-to-Big Retrieval（Sentence Window）

適合情境：

- 長文件
- 查詢精細事實

作法：

- Index 用單句 embedding
- 回傳時補上上下文 window (`MetadataReplacementPostProcessor`)

![caption](Gemini_Generated_Image_f4triqf4triqf4tr.png)

In [ ]:
from glob import glob
from pathlib import Path

from llama_index.core import SimpleDirectoryReader

wiki_metadatas = {
    "鋼之鍊金術師": {
        "author": "荒川弘",
    },
    "一拳超人": {
        "author": "ONE",
    },
    "ONE_PIECE": {
        "author": "尾田榮一郎",
    },
    "東京喰種": {
        "author": "石田翠",
    },
}

docs = SimpleDirectoryReader(input_files=[f for f in glob("data/*.txt")]).load_data()

for doc in docs:
    comic_name = Path(doc.metadata['file_name']).stem
    doc.metadata.update(wiki_metadatas[comic_name])

In [ ]:
from llama_index.core.node_parser.text.sentence_window import SentenceWindowNodeParser
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

sw_node_parser = SentenceWindowNodeParser(
    # How many sentences on both sides to capture. 
    # Setting this to 3 results in 7 sentences.
    window_size=3, # # 前後各 3 句，共 7 句
    # the metadata key for to be used in MetadataReplacementPostProcessor
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence"
)

sw_nodes = sw_node_parser.get_nodes_from_documents(docs)

In [ ]:
import os

os.chdir("../")

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex

from src.ollama_connection import llama_index_ollama
from src.storage_context_init import init_faiss_storage_context

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

sw_storage_context = init_faiss_storage_context(d=1024)
ollama_llm = llama_index_ollama(model="gpt-oss:120b-cloud", temperature=0)

sw_index = VectorStoreIndex(
    sw_nodes,
    storage_context=sw_storage_context,
    embed_model=embed_model,
)

In [ ]:
sw_query_engine = sw_index.as_query_engine(
    similarity_top_k=5,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
    llm = ollama_llm
)

In [ ]:
output = sw_query_engine.query(
    "最強的光頭是誰?"
)
print(output.response)

## Auto-Retrieval

當資料規模變大時：

- 單純靠相似度搜尋 → 雜訊急遽增加
- Metadata 可以先縮小搜尋子空間

VectorIndexAutoRetriever 的核心思想是：

User Query
→ LLM 推斷可能的 metadata 條件
→ 套用 filters 進行向量搜尋
→ 回傳更精準的節點

這讓 Retrieval 從「被動比對」進化成「語意驅動的檢索策略」。

** 無法搭配 FAISS **

In [ ]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo


vector_store_info = VectorStoreInfo(
    content_info="作者個人資料",
    metadata_info=[
        MetadataInfo(
            name="author",
            type="str",
            description=(
                "漫畫作者"
            ),
        ),
    ],
)

In [ ]:
await aclient.close()

In [ ]:
from qdrant_client import AsyncQdrantClient
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant_autoretrieval")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    vector_store=qdrant_vector_store,
)

autoretrieval_index = VectorStoreIndex(
    sw_nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    use_async=True
    )

In [ ]:
retriever = VectorIndexAutoRetriever(
    autoretrieval_index,
    vector_store_info=vector_store_info,
    llm=ollama_llm,
    max_top_k=1000,
    similarity_top_k=5
)

In [ ]:
# async模式
filtered_nodes = await retriever.aretrieve(
    "尾田榮一郎的作品有哪些?"
)

In [ ]:
# filtered_nodes

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

# 將你的 retriever 封裝進 Query Engine
query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    llm=ollama_llm  # 建議再次傳入 llm 以確保合成答案時使用相同的模型
)

# 測試調用
output = await query_engine.aquery("尾田榮一郎的作品有哪些?")
print(output.response)

## HierarchicalNodeParser

此節點解析器會將節點切分成 **階層式節點（hierarchical nodes）**。  
也就是說，單一輸入會被切分成多個不同大小的階層，每個節點都會包含對其 **父節點** 的參考。

當與 `AutoMergingRetriever` 結合使用時，這可以讓我們在大多數子節點被檢索到時，自動將檢索到的節點替換為其父節點。  
這個過程可以為 LLM 提供更完整的上下文，有助於生成更精確的回答。

| 步驟 | 動作 | 說明 |
|------|------|------|
| 1 | 檢索 | 找尋 Leaf Nodes，系統在最細的層級（Leaf）尋找語義最相關的碎片。 |
| 2 | 計數 | 檢查父節點覆蓋率，系統會檢查：「某個父節點下的子節點，是否被檢索到了超過 50%（或其他閾值）？」 |
| 3 | 合併 | 自動向上合併，如果條件滿足，系統會捨棄這些散亂的小子節點，直接抓取整個父節點提供給 LLM。 |


為什麼這很有用？
- 避免「斷章取義」： 單獨一個 128 tokens 的句子可能遺失了前因後果。
- 動態上下文： 如果只有一小句話相關，就給一小句（節省 Token）；如果一整段都相關，就給一整段（增加精確度）。
- 範例場景： 如果你問「這家公司的退貨政策是什麼？」，系統檢索到了「退貨期限」、「退貨運費」、「退貨包裝」三個小子節點。與其把這三段碎碎念給 LLM，AutoMergingRetriever 會直接把整個完整的「退貨政策大章節」丟給它。

#### 參考資料
- [知乎專欄文章](https://zhuanlan.zhihu.com/p/678698001)

In [ ]:
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes, get_root_nodes

hierarchical_node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

hierarchical_nodes = hierarchical_node_parser.get_nodes_from_documents(docs)

In [ ]:
len(hierarchical_nodes)

In [ ]:
leaf_nodes = get_leaf_nodes(hierarchical_nodes)
root_nodes = get_root_nodes(hierarchical_nodes)

print(f"# of leaf nodes: {len(leaf_nodes)}")
print(f"# of root nodes: {len(root_nodes)}")

**為何總數是 990？**

當你設定：

```python
chunk_sizes = [2048, 512, 128]
```

解析器會針對同一段文本生成三個層級的節點：

- 第一層（Root）：最大區塊（2048 tokens）
- 第二層（Mid）：將每個 2048 tokens 的區塊再細分為數個 512 tokens 的區塊
- 第三層（Leaf）：將每個 512 tokens 的區塊再細分為數個 128 tokens 的區塊

hierarchical_nodes 會包含這三個層級所有節點的總和，因此：

$$
    總節點數 (990) = Root 節點 + 中間層節點 + Leaf 節點
$$


**節點類型的定義**

Root Nodes（根節點 - 32 個）

- 代表文本的最上層結構（2048 tokens）
- 意味著你的原始文件 docs 被切成 32 個大型區塊

Leaf Nodes（葉節點 - 803 個）

- 最底層、最小單位的區塊（128 tokens）
- 在 Retrieval（檢索） 時通常會優先使用
- 原因是語義最精確、最容易匹配使用者問題

中間節點（512 tokens，未直接印出）

- 在你的例子中也存在一層 512 tokens 的中間節點
- 可由總數推算得出：

**為什麼要這樣設計？**

這種分層結構是為了實現 Auto-Merging Retrieval（自動合併檢索）。

🎯 精準檢索（Fine-grained Retrieval）

- 向量搜尋時使用最小的 Leaf Nodes（128 tokens）
- 碎片化的資訊更容易命中使用者的查詢語義

🧩 上下文還原（Context Reconstruction）

- 當系統發現某個父節點（512 或 2048 tokens）下
- 大多數子節點被選中
- 系統會「向上合併」
- 直接將整個父節點內容提供給 LLM

⚖️ 解決精準度與上下文的矛盾

- 小 chunk：提高搜尋精準度
- 大 chunk：保留完整語意與上下文
- Auto-Merging 讓兩者兼得

In [ ]:
await aclient.close()

In [ ]:
aclient = AsyncQdrantClient(path="week_2/langchain_qdrant_hierarchical")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    vector_store=qdrant_vector_store,
)

storage_context.docstore.add_documents(hierarchical_nodes)

leaf_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    use_async=True
    )

**AutoMergingRetriever 為何能找回父節點？**

在程式碼中，**雖然 `VectorStoreIndex` 只接收了 `leaf_nodes`**，  
但 `AutoMergingRetriever` 卻能成功找回 **父節點（甚至根節點）**。

這個「看似魔法」的能力，其實來自於：

- `storage_context`
- 節點本身的 `relationships`（血緣關係）

以下我們用 **三個核心步驟** 拆解它的運作原理。

---

**1. 節點中的「血緣關係」（Relationships / Metadata)**

當你執行：

```python
hierarchical_nodes = hierarchical_node_parser.get_nodes_from_documents(docs)
```

LlamaIndex 不只是切分文字，還在每個節點的 metadata 中注入了 relationships：

- 每個 Leaf Node（葉節點）
- 都會保存一個指向其 Parent Node（父節點） 的 parent_id

Leaf Node A
 
 └── parent_id: "Middle_Node_001"

Middle_Node_001
 
 └── parent_id: "Root_Node_001"

**2. Docstore：節點的「保險箱」**

storage_context 在整個流程中扮演了關鍵角色：
```
hierarchical_retriever = AutoMergingRetriever(
    leaf_retriever,
    storage_context,  # 關鍵就在這裡！
    verbose=True
)

```
為什麼 storage_context 這麼重要？

- VectorStoreIndex
    👉 只 將 leaf_nodes 的向量存入 Qdrant

- storage_context.docstore
    - 👉 保存所有節點物件
    - Leaf Nodes
    - Middle Nodes
    - Root Nodes（即使它們沒有被向量化）

📦 Docstore 就像一個完整的文件保險箱，裡面存放著所有層級的節點內容與關係。

**3. AutoMergingRetriever 的自動合併流程**

當你發起查詢時，AutoMergingRetriever 會依序執行以下步驟：

- Step 1：檢索葉節點（Vector Search）

    - leaf_retriever
    - 從 Qdrant 中找出最相似的 N 個 Leaf Nodes（例如：6 個）
---

- Step 2：追溯父節點（Docstore Lookup）

    - 讀取這些 Leaf Nodes 的 parent_id
    - 前往 storage_context.docstore
    - 取回對應的 父節點物件
---
- Step 3：檢查是否符合「合併條件」

    - Retriever 會計算：

      「這個父節點底下的子節點，有多少比例被檢索到了？」

    - 如果符合門檻（例如：

        - 超過一定比例

        - 或至少多個子節點命中）

    ➡️ 判定這些碎片資訊 足以代表整個區塊

---

- Step 4：替換節點（Auto Merge）

    - 丟棄零碎的 Leaf Nodes
    - 改為回傳 完整的父節點（甚至根節點）內容
    - 將更有上下文的文字送給 LLM

In [ ]:
from llama_index.core.retrievers import AutoMergingRetriever

leaf_retriever = leaf_index.as_retriever(similarity_top_k=6, embed_model=embed_model)
hierarchical_retriever = AutoMergingRetriever(leaf_retriever, storage_context, verbose=True)

automerging_query_engine = RetrieverQueryEngine.from_args(hierarchical_retriever, llm=ollama_llm)

## SummaryIndex

在介紹接下來的 **RecursiveRetriever** 與 **IndexNode** 組合之前，先來說明一下 **SummaryIndex** 的概念與用途。

**SummaryIndex** 的核心特性在於：  
它會「一次性地消化所有 nodes」，將整份資料視為一個整體來處理，而不是逐條或逐層地進行檢索。

你可以把它想像成 **泰倫蟲族（Tyranids）**——  
不是慢慢啃食單一目標，而是直接吞噬整個生態系，並將其轉化為可用的生物質。  
差別只在於，SummaryIndex 消化的不是血肉，而是「語意」。

---

### 運作方式概念

SummaryIndex 的運作流程可以簡化為以下幾個步驟：

1. **讀取所有 nodes**  
   不論資料大小或層級，所有節點都會被納入處理範圍。

2. **整合與壓縮語意**  
   透過 LLM 將分散在各個 nodes 中的資訊進行整合，去除重複細節，保留關鍵概念。

3. **產出高層次摘要**  
   最終得到的是一個（或少數幾個）代表整體內容的摘要節點，而非可直接對應回原始細節的片段。

這使得 SummaryIndex 本質上是一種 **「全局視角」的 Index**。

---

### 適合的使用場景

SummaryIndex 特別適合回答以下類型的問題：

- 這份文件「整體在講什麼」？
- 這個系統或專案的「設計核心」是什麼？
- 從全局來看，有哪些「關鍵概念與主軸」？

它並不追求精準命中某一小段內容，而是提供對整體語意的理解。

---

### 與其他 Index 的差異

將 SummaryIndex 與常見的 Vector-based Index 作一個對比：

- **Vector Index**
  - 著重於語意相似度搜尋
  - 目標是找出「最相關的片段」
  - 適合精細查詢與事實定位

- **SummaryIndex**
  - 著重於整體語意整合
  - 目標是「用最少的內容代表全部」
  - 適合總覽、摘要與高層理解

換句話說，Vector Index 回答的是「哪一段最相關」，  
而 SummaryIndex 回答的是「如果只能說一次，該怎麼說完全部內容」。

---

### 為什麼要先介紹 SummaryIndex？

理解 SummaryIndex 這種「一次性消化」的策略，對後續理解  
**RecursiveRetriever + IndexNode** 的設計非常關鍵。

- SummaryIndex 提供的是 **自上而下的整體抽象**
- RecursiveRetriever 負責 **逐層展開、逐步深入**
- IndexNode 則用來 **串接不同層級的語意結構**

你可以將 SummaryIndex 視為整個知識結構的「頂層總覽」，  
而後續的組合設計，則是在這個總覽之下，逐步向細節層級鑽探。


In [ ]:
import wikipediaapi
from langchain_text_splitters import RecursiveCharacterTextSplitter
from llama_index.core import SummaryIndex, get_response_synthesizer, Document
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

wiki_wiki = wikipediaapi.Wikipedia(user_agent='AI Tutorial(mengchiehling@gmail.com)', language='zh-tw')

title = "银之匙_Silver_Spoon"

page = wiki_wiki.page(title)
wiki_text = page.text

new_doc = Document(text=wiki_text, metadata={"author": "荒川弘"})

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=128,
    chunk_overlap=0,
    separators=["\n\n", "\n", "。", "，", " "]
).split_text

semantic_splitter_node_parser = SemanticSplitterNodeParser.from_defaults(
    embed_model = embed_model,
    sentence_splitter=text_splitter,
    include_metadata=True,
    include_prev_next_rel=True,
    breakpoint_percentile_threshold=60,
)

new_nodes = semantic_splitter_node_parser.get_nodes_from_documents([new_doc])

In [ ]:
basic_summary_index = SummaryIndex(new_nodes)
 = basic_summary_index.as_query_engine(llm=ollama_llm, response_mode='compact',
                                       embed_model=embed_model)

In [ ]:
output = await basic_summary_query_engine.aquery("給我一個銀之匙的快速簡介")

In [ ]:
print(output.response)

### response_mode

直接在 Query Engine 設定（簡單）
```
query_engine = index.as_query_engine(
    llm=ollama_llm,
    response_mode="tree_summarize"
)
```

使用 get_response_synthesizer（進階自定義

為什麼要手動使用 get_response_synthesizer？
如果你想更精細地控制「合成答案」的過程（而不僅僅是改個模式名），你可以手動建立它。這讓你能夠自定義：

- Prompt Templates: 自定義如何提問。
- Output Parsing: 格式化輸出。
- Streaming: 是否開啟串流顯示。

```
from llama_index.core import get_response_synthesizer

# 1. 建立 synthesizer
response_synthesizer = get_response_synthesizer(
    llm=ollama_llm,
    response_mode="compact",
    streaming=True, # 額外的設定
    # text_qa_template=my_custom_prompt # 甚至可以自定義 Prompt
)

# 2. 將其注入 query_engine
query_engine = index.as_query_engine(
    response_synthesizer=response_synthesizer
)
```

| 模式名稱 | 運作邏輯 | 適用場景 |
|---------|----------|----------|
| refine（預設） | 逐一處理節點。先用第一個節點生成答案，再將該答案與下一個節點交給 LLM 進行「修正 / 精煉」。 | 想要最完整、最精確的答案，不計較 API 呼叫次數。 |
| compact | 類似 refine，但在處理前會先將多個節點合併到同一個 Prompt 中，以節省 Token 和 API 呼叫次數。 | 推薦使用。平衡了效率與品質。 |
| tree_summarize | 將節點分組並生成摘要，再對摘要進行摘要，最終形成一棵樹狀結構直到得出最後答案。 | 適合總結整份文件或處理跨多處的資訊。 |
| simple_summarize | 直接將所有節點塞進一個 Prompt 中。如果節點太多超過 Context Window 會報錯。 | 節點數量少且希望快速得到答案時。 |
| no_text | 僅返回檢索到的節點（Source Nodes），不呼叫 LLM 生成回答。 | 用於測試檢索準確度或僅需原始資料時。 |
| accumulate | 對每個節點獨立生成一個答案，最後將所有答案拼接在一起返回。 | 需要針對不同段落分別給出回答時。 |


## RecursiveRetriever 與 IndexNode

隨著以 LLM 為核心的系統從小型、扁平的文件集合，擴展到更複雜的知識庫，單純依賴相似度搜尋的檢索方式已逐漸不足以應付實際需求。真實世界的資料往往具有 **階層性（hierarchical）**、**模組化（modular）** 與 **多層次粒度（multi-granular）** 的特性：書籍由章節與段落組成、知識庫依領域劃分，而摘要自然位於詳細內容之上。  
LlamaIndex 正是透過 **RecursiveRetriever** 與 **IndexNode** 來解決這類問題。

### IndexNode

**IndexNode** 是一種特殊的節點（node），它不僅包含文字內容，還 **指向另一個索引（index）或檢索器（retriever）**。從概念上來看，它是一座連接不同抽象層級的「橋樑」。

你可以將 IndexNode 想成：
- 一段摘要、標籤或描述文字（例如：「第 2 章：檢索方法」）
- 再加上一個指向更深層索引的參考

這使得節點可以代表 **一整組節點的集合**，而不只是單一的文字片段。與其將所有低層級的內容一次性嵌入到一個龐大的索引中，不如先嵌入高層摘要，並由它們連結到更細節的索引。

### RecursiveRetriever

**RecursiveRetriever** 是一種可以同時處理一般節點與 IndexNode 的檢索器。當它檢索到一個 IndexNode 時，流程並不會就此停止，而是會 **遞迴地呼叫該 IndexNode 所指向的檢索器**，並在下一層繼續進行檢索。

在高層次上，整個檢索流程如下：

1. 先檢索高層節點（通常是摘要或分類）
2. 在結果中辨識出 IndexNode
3. 依照 IndexNode 的指向，進入更深層的檢索器或索引
4. 檢索更細粒度的節點
5. 重複上述流程，直到抵達最底層的節點

這樣的設計讓檢索不再只是單次的平面搜尋，而是 **多階段、具階層性的檢索過程**。

### 為什麼這很重要

將 RecursiveRetriever 與 IndexNode 搭配使用，可以讓你：
- 在不一次嵌入所有資料的情況下，擴展到大型語料庫
- 保留文件原有的結構與階層關係
- 實現「由粗到細」的檢索流程（由整體概覽逐步深入細節）
- 僅在相關的子空間中搜尋，降低雜訊
- 組合不同類型的索引（例如：SummaryIndex → VectorStoreIndex）

### 心智模型（Mental model）

- **IndexNode**：「這段文字代表一個集合，可以從這裡往下深入。」
- **RecursiveRetriever**：「先檢索，再順著連結一路往下，直到找到合適的細節層級。」

兩者結合，構成了 LlamaIndex 中 **結構化、階層式檢索管線** 的核心基礎，使你能打造更精準且高效率的 RAG 系統。

假如你是一個鎚佬:

| 技術概念               | 戰鎚 40K 比喻     |
| ------------------ | ------------- |
| RecursiveRetriever | 神聖泰拉 / 帝國行政中樞 |
| Query（查詢）          | 帝皇或高領主的命令     |
| IndexNode          | 登記在案的星球       |
| Node 指向子 Index     | 星區 → 子星區 → 行星 |
| 檢索子索引              | 徵調十一稅         |
| 回傳結果               | 資源／軍團／數據送回泰拉  |



In [ ]:
from glob import glob
from pathlib import Path

import faiss
from llama_index.core import SummaryIndex, SimpleDirectoryReader
from llama_index.core.schema import IndexNode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.faiss import FaissVectorStore

In [ ]:
# define top-level nodes and vector retrievers
index_nodes = []
vector_query_engines = {}
vector_retrievers = {}

wiki_metadatas = {
    "鋼之鍊金術師": {
        "author": "荒川弘",
    },
    "一拳超人": {
        "author": "ONE",
    },
    "ONE_PIECE": {
        "author": "尾田榮一郎",
    },
    "東京喰種": {
        "author": "石田翠",
    },
}

docs = SimpleDirectoryReader(input_files=[f for f in glob("week_2/data/*.txt")]).load_data()

d = 1024
faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)

sentence_splitter_node_parser = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)

for doc in docs:
    comic_name = Path(doc.metadata['file_name']).stem
    doc.metadata.update(wiki_metadatas[comic_name])
    storage_context = StorageContext.from_defaults(
        vector_store=vector_store.model_copy(),
    )
    
    # 建立index
    nodes = sentence_splitter_node_parser.get_nodes_from_documents([doc])
    vector_index = VectorStoreIndex(
        nodes,
        embed_model=embed_model,
        storage_context=storage_context
    )
    
    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=ollama_llm, similarity_top_k=5)
    vector_query_engines[comic_name] = vector_query_engine
    vector_retrievers[comic_name] = vector_index.as_retriever()

    # save summaries
    out_path = Path("week_2/summaries") / f"{comic_name}.txt"
    if not out_path.exists():
        # use LLM-generated summary
        summary_index = SummaryIndex(nodes)

        summarizer = summary_index.as_query_engine(
            response_mode="compact", llm=ollama_llm
        )
        response = await summarizer.aquery(
            f"請給我{comic_name}的總結"
        )

        wiki_summary = response.response
        Path("week_2/summaries").mkdir(exist_ok=True)
        with open(out_path, "w", encoding="utf-8") as fp:
            fp.write(wiki_summary)
    else:
        with open(out_path, "r", encoding="utf-8") as fp:
            wiki_summary = fp.read()

    print(f"**Summary for {comic_name}: {wiki_summary}")
    node = IndexNode(text=wiki_summary, index_id=comic_name)
    index_nodes.append(node)

In [ ]:
top_vector_index = VectorStoreIndex(
    index_nodes, embed_model=embed_model
)

top_vector_retriever = top_vector_index.as_retriever(similarity_top_k=5)

In [ ]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": top_vector_retriever, **vector_retrievers},
    # query_engine_dict=vector_query_engines,
    verbose=True,
)

| 參數                    | 說明                                                                                         | 範例                                                                              |
| --------------------- | ------------------------------------------------------------------------------------------ | ------------------------------------------------------------------------------- |
| **root_id**           | 字串，指定最上層（root）的 retriever ID                                                               | `"vector"` → 就是 `retriever_dict["vector"]`                                      |
| **retriever_dict**    | 一個字典，key 是 retriever ID，value 是實際的 Retriever。<br>用來定義每個節點對應的 retriever                     | `{ "vector": top_vector_retriever, "鋼之鍊金術師": steel_alchemist_retriever, ... }`  |
| **query_engine_dict** | 可選。指定每個 retriever 對應的 **QueryEngine**（有時包含 LLM 回答能力）。<br>如果不提供，Retriever 只會做「檢索」，不會直接生成回答。 | `{ "vector": top_vector_query_engine, "鋼之鍊金術師": steel_alchemist_query_engine }` |
| **verbose**           | 是否在檢索/路徑選擇時輸出 debug 訊息                                                                     | `True/False`                                                                    |

🔹 為什麼 `root_id` 是 `"vector"`？

`RecursiveRetriever` 需要知道「最上層從哪裡開始」：

- `"vector"` 對應到 `retriever_dict["vector"]`，也就是你的 **summary index**  
- Retriever 會先在 summary index 找出最相關的節點，再根據 recursive mapping 去下層檢索 full-text

---

🔹 為什麼要 `retriever_dict`？

這就是「兩層結構的核心」：

- key → retriever 的 ID  
- value → retriever 物件（可以是 VectorRetriever、BM25Retriever 等）

> 這樣 RecursiveRetriever 才知道：當 top 層選到 `"一拳超人"` 時，要往哪個下層 retriever 查。

---

🔹 `query_engine_dict` 的作用

1. **Retriever 只做檢索**  
   - 會回傳節點或 chunk  
   - 沒有 LLM 生成回答能力  

2. **QueryEngine 結合 Retriever + LLM**  
   - 可以直接用 LLM 生成回答或摘要  
   - RecursiveRetriever 如果有 `query_engine_dict`，就可以直接呼叫 QueryEngine 做「最終回答」而不是只回 chunk

> 簡單比喻：
> - `retriever_dict` = 找書的書架  
> - `query_engine_dict` = 書架 + 讀書的人 → 可以直接講故事給你聽

---

🔹 何時需要 `query_engine_dict`？

- 你希望 recursive retriever **每一層都用 LLM 生成回答**  
- 如果只是純向量檢索，`query_engine_dict` 可以不用  

例子：

```python
recursive_retriever = RecursiveRetriever(
    root_id="vector",
    retriever_dict={"vector": top_vector_retriever, **vector_retrievers},
    query_engine_dict=vector_query_engines,  # 每個 retriever 對應 QueryEngine
    verbose=True,
)
```
- 這樣 recursive_retriever.get_relevant_documents(query) 會自動：
    1. Top 層用 summary 找到對應漫畫
    2. 下層 retriever 找 chunk
    3. QueryEngine 用 LLM 整理答案


In [ ]:
recursive_query_engine = RetrieverQueryEngine.from_args(
    retriever=recursive_retriever,
    llm=ollama_llm,
    response_mode='compact'
)

### 回家挑戰

如何透過Persist重構RecursiveRetriever